# Cell segmentation in byc
The workflow for segmentation is a combination of manual annotation in imagej assisted by plugins and macros in `imagejpc` and data processing in python with functions from `byc.segmentation`

## ImageJ scripts (imagejpc) installation 
### Clone `imagejpc` repo

```sh
git clone https://github.com/johnpcooper/imagejpc.git
tree imagejpc
imagejpc
├── Keyboard_Shortcuts.xls
├── README.md
├── macros
│   ├── create_and_measure_rois.ijm
│   ├── scale_rois.ijm
│   └── threshold_brightfield.ijm
└── utilities
    ├── get_percentile.py
    ├── measure_and_clear_rois.py
    ├── measure_background.py
    ├── measure_rois.py
    ├── measure_steady_state.py
    ├── record_cell.py
    ├── save_cell_measurement_rois.py
    ├── save_cell_roi_set.py
    ├── save_rois.py
    ├── save_tifs.py
    ├── split_and_save_stack.py
    └── time_stamper_jpc.py
```

### Get plugins and macros installed in `imagejpc`
1. Install [Fiji](https://imagej.net/Fiji/Downloads) if not already installed.


2. Open imagej, click Plugins > Macros > Install > Select macros (one at a time, ugh) from path/to/imagejpc/macros


3. Click Plugins > Install PlugIn > Select .py files (one at a time) from path/to/imagejpc/utilities


4. Click Plugins > Shortcuts > Add Shortcut... to make shortcut for shortcuts in `Keyboard_Shortcuts.xls`


4. Restart imagej to complete installation

## Creating cell ROIs in Imagej
Use Imagej to annotate cell locations and bud appearance timepoints on aligned FOV stacks (one for each channel collected in the experiment

An experiment directory should look something like

```sh
tree 20200221_byc_analysis
20200221_byc_analysis
├── 20200221_byc_analysis
├── 20200221_byc_master_index.csv
├── 20200221_byc_notes.txt
└── pJC069_rpn4d
    ├── 20200221_byc_xy00_bf_stack.tif
    ├── 20200221_byc_xy00_dsred_stack.tif
    ├── 20200221_byc_xy00_yfp_stack.tif
    ├── 20200221_byc_xy01_bf_stack.tif
    ├── 20200221_byc_xy01_dsred_stack.tif
    └── 20200221_byc_xy01_yfp_stack.tif
```

`20200221_byc_master_index.csv` is an index with a different row for each cell and information about where to find that cell's ROIs, which xy it was found in, where to find measurement .csvs etc. 

### Crop (tracking) ROIs
These ROIs will be used in `byc.segmentation` to crop cells from each channel in the cell's source FOV stack and threshold a fluorescent channel chosen by the user. 

1. Open a brightfield and/or fluorescent channels FOV stack(s) in Imagej


2. To create a crop ROI, draw a rectangular selection around the mother cell to be tracked and press t to add it to RoiManager


3. Each ROI should have the mother cell approximately centered and should exclude any other cells in the same catch tube. 


4. First ROI should be when cell first appears in experiment


5. Make a new ROI every time the cell moves out of the center of the ROI (whether because a division moves it or a registration issue)


6. Last ROI should be either last frame cell was seen alive, or whenever the user wants to end trace


7. Once you've finished adding crop ROIs to RoiManager, press l to focus the Imagej search bar, then type 'save cell roi set'. Type 'crop' for roi set type and whatever the end event type was. Information about this cell then gets added to the results table and you can do what you want with that. I will eventually merge these results with the `master_index.csv` to streamline information gathering

The `save_cell_roi_set.py` plugin will save the crop set in the active image directory with:

```python
roi_set_fn = '{}_cell{}_{}_rois'.format(expt_title, cell_index, roi_set_type)
```

`cell_index` is a three digit number (eg 001) generated by looking in the active image directory, and counting the number of `.roi` or `.zip` (ROI format) files in that directory. So if you have bud or measurement ROI.zip or ROI.roi files in that directory you should temporarily move them out so that the cell count is accurate.

### Measurement (cell outline) ROIs
Use python to crop crop ROIs out of each channel FOV stack and threshold the fluorescent channel used for segmentation. Then open the cropped cell stack's in Imagej and run more imagejpc plugins to segment the cell outline and save fluorescence measurements.

If you've made crop ROIs for one cell (steps above), then your experiment directory should now look like:

```sh
tree 20200221_byc_analysis
20200221_byc_analysis/
├── 20200221_byc_analysis
├── 20200221_byc_master_index.csv
├── 20200221_byc_notes.txt
└── pJC069_rpn4d
    ├── 20200221_byc_cell000_crop_rois.zip
    ├── 20200221_byc_xy00_bf_stack.tif
    ├── 20200221_byc_xy00_cell000_bf_stack.tif
    ├── 20200221_byc_xy00_cell000_dsred_otsu_stack.tif
    ├── 20200221_byc_xy00_cell000_dsred_stack.tif
    ├── 20200221_byc_xy00_cell000_yfp_stack.tif
    ├── 20200221_byc_xy00_dsred_stack.tif
    ├── 20200221_byc_xy00_yfp_stack.tif
    ├── 20200221_byc_xy01_bf_stack.tif
    ├── 20200221_byc_xy01_dsred_stack.tif
    └── 20200221_byc_xy01_yfp_stack.tif
```

#### Workflow for creating measurement ROIs:

1. Open each cell stack in imagej, eg:
    * 20200221_byc_xy00_cell000_bf_stack.tif
    * 20200221_byc_xy00_cell000_dsred_otsu_stack.tif
    * 20200221_byc_xy00_cell000_dsred_stack.tif
    * 20200221_byc_xy00_cell000_yfp_stack.tif
    

2. Tab to the otsu thresholded cell stack (binary image) and press g, select Method: Default, Background: Default, and check no boxes, then press enter. Cell area should black and background should be white.


3. Then use Imagej's Analyze Particles functionality to find cell area using the otsu thresholded cell stack (this should be rolled into a script):
    * Press 8 to run Watershed
    * Press q to Set Measurements. Make sure Area, integrated density, Mean gray value, Perimeter, Fit ellipse, Feret's diameter, Median, Stack position, and Display label are all checked. Then hit OK.
    * Press u to Analyze Particles. Typically set Size to 70-1000 pixels if 60X objective. Circularity 0.55-1.00. Show nothing. Check only Add to Manager. Hit OK
    

4. Make sure there's an ROI for the first and last frame. Make sure that there are ROIs for each significant position and size/shape change of the cell so that interpolation will actually stay with the cell. Draw any new ROIs manually or use the shape from next ROI and add to RoiManager by pressing t. You should have something like this (but not necessarily an ROI for every frame):

<img src="https://previews.dropbox.com/p/thumb/AA2yjtuxbUOFRK1lEkILrIA9JYV_gneV9gg93xNvmpP6UE2GYERw_CJQyngmo3OQA6RPMgNqecruQqK36IgkULMXctMQ2hKmFDtchQ5bpvz5XnA4Du2svjkGsFxjhi1rkUd2kx7Aueqy79ZKmyu1E76BvDVC6ssNFCAj1Xi_7m5jSyMGOXsgu6i9I7JrPuwsFm6TQFa2ge7XqIeYw9jfaJWKPdO8D5wGIqccnwUdIUuvmEkG6mDNS3exsTLE7KODmsCplhMdYn3wfVbHlADu6xeqS7dRvLnLG7OZJOWRTPHDSFt8KIlA5jxW1XHAa-UDgRPz9hlRDj1wgpTJqINgzfFU3oLPmH3sjrkY0GPg-tfhfA/p.png" width=400/>


5. Press l to focus the search bar and run 'save rois'. `save_rois.py` will sort the ROIs, make sure there's an ROI at first and last frame, then interpolate the ROIs. Then it will check if any frames have > 1 ROI. If not, the ROI set will be saved according to the active image title


6. Once you've successfully saved an roi set using 'save rois', the cell stacks are ready to measure. Focus the search bar with l then type 'measure rois'. `measure_rois.py` will make and save meausurements automatically.

The experiment directory should now look like:

```sh
20200221_byc_analysis/
├── 20200221_byc_master_index.csv
├── 20200221_byc_notes.txt
└── pJC069_rpn4d
    ├── 20200221_byc_cell000_crop_rois.zip
    ├── 20200221_byc_xy00_bf_stack.csv
    ├── 20200221_byc_xy00_bf_stack.tif
    ├── 20200221_byc_xy00_cell000_bf_stack.csv
    ├── 20200221_byc_xy00_cell000_bf_stack.tif
    ├── 20200221_byc_xy00_cell000_bf_stack_measurement_rois.zip
    ├── 20200221_byc_xy00_cell000_dsred_otsu_stack.tif
    ├── 20200221_byc_xy00_cell000_dsred_stack.csv
    ├── 20200221_byc_xy00_cell000_dsred_stack.tif
    ├── 20200221_byc_xy00_cell000_yfp_stack.csv
    ├── 20200221_byc_xy00_cell000_yfp_stack.tif
    ├── 20200221_byc_xy00_dsred_stack.tif
    ├── 20200221_byc_xy00_yfp_stack.tif
    ├── 20200221_byc_xy01_bf_stack.tif
    ├── 20200221_byc_xy01_dsred_stack.tif
    └── 20200221_byc_xy01_yfp_stack.tif
```

### Bud ROIs

Bud ROIs are created to manually track the timepoint (frame number) when daughter cells are born from a tracked mother

#### Workflow for creating Bud ROIs

1. Open a brightfield FOV stack in Imagej


2. Draw a rectangular selection around the cell. Press t to add the selection to RoiManager as an ROI


3. First ROI should be the first frame in which the mother cell being tracked appears


4. Intermediate ROIs should be the frame at which a bud first emerges from the mother cell or the frame before


5. Last ROI should be the frame the mother on which the mother was last seen alive


6. Press l to focus the Imagej search bar. Type 'save cell roi set' and press enter. Imagej will ask for end event type which can be:
    * 'death'
    * 'escape'
    * 'sen' (Means the cell appeared senescent (produced its last daughter cell) at the end of data collection but still hasn't lysed)
    
The `save_cell_roi_set.py` plugin will then save the bud roi set in the active image directory